# Install Dependencies

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%%time
! pip install transformers datasets accelerate -q
! pip install sentence-transformers -q
! pip install faiss-gpu -q

# ! pip install deep-translator -q
# ! pip install peft -q
# ! pip install bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.8 MB/s eta 0:00:00
CPU times: user 152 ms, sys: 6.5 ms, total: 158 ms
Wall time: 21.8 s


In [ ]:
import torch
import pandas as pd
import tqdm
import numpy as np
import pickle
import datasets
import re
import json

from collections import Counter
import faiss
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM

# Helper Function

In [ ]:
def convert_name_NER_batch(examples):
    model_name = '/content/drive/MyDrive/NER_Model'
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding='longest', max_length=108, truncation=True, use_fast=True)
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    ner = pipeline("ner", model=model, tokenizer=tokenizer)

    question_batch = examples['question']
    result_batch = ner(question_batch)

    ner_question_batch = []
    for result, question in zip(result_batch, question_batch):
        i = 1
        person_dict = {}
        for ent in result:
            ent_label = ent['entity']
            ent_word = ent['word']

            if ent_label == 'B-PERSON':
                if ent_word not in person_dict:
                    replace_name = 'Person' + str(i)
                    person_dict[ent_word] = replace_name
                    i += 1
        for k, v in person_dict.items():
            question = question.replace(k, v)

        ner_question_batch.append(question)

    examples['ner_question'] = ner_question_batch

    return examples

In [ ]:
def translate_dataset_batch(examples):  # Faster to use in batch
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model_name = '/content/drive/MyDrive/Model_Translation_envit5'
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    columns = ['ner_question','option_1','option_2','option_3','option_4','option_5']
    translated_dataset = {}

    for col in columns:
        if col == 'ner_question':
            inputs = [f'vi: {text}' for text in examples[col]]

        else:
            inputs = ['vi: ' + re.sub(r'[A-E]\.', '', item).strip() for item in examples[col]]

        inputs = tokenizer(inputs, return_tensors="pt", padding="longest", truncation=True, max_length=108)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Translate the inputs in batches
        with torch.inference_mode():
            outputs = model.generate(**inputs, min_length=0,max_length=108,no_repeat_ngram_size=2)

        # Decode the outputs and remove the special tokens
        translated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        trans_texts = [text.replace('en:','').replace('vi:','').strip() for text in translated_texts]

        translated_dataset[f'trans_{col}'] = trans_texts

    return translated_dataset

# Sentence Embedding Corpus

## Load file embeddings (no need to run)

In [ ]:
with open('/content/drive/MyDrive/embeddings_430_231111.pkl', 'rb') as file:
    numpy_embed = pickle.load(file)

In [ ]:
numpy_embed.shape

(2982, 3)

In [ ]:
numpy_embed_content = numpy_embed.rename(columns={'Combined_Content_2': 'Content'}).drop(columns='Embeddings')
numpy_embed_content.head(3)

,Url,Content
0,alzheimer,What is Alzheimer's disease? Alzheimer's is a ...
1,alzheimer,"Reason. Therefore, the brain is considered a g..."
2,alzheimer,Common symptoms of Alzheimer's syndrome. The s...


In [ ]:
numpy_embed_content.to_csv('df_768_break.csv',index=False)

# Text Retriever

## Save faiss index (do not run, just run when set up 1 time)

In [ ]:
# with open('/content/drive/MyDrive/df_final_768.pkl', 'rb') as file:
#     df_final = pickle.load(file)

In [ ]:
import faiss
dimension = 768
index = faiss.IndexFlatL2(dimension)

In [ ]:
embeddings = np.vstack(numpy_embed['Embeddings'].values)

In [ ]:
embeddings.shape

(2982, 768)

In [ ]:
index.add(embeddings)

In [ ]:
index_file_path = '/content/drive/MyDrive/faiss_768_break.index'
# Save the index to the specified file
faiss.write_index(index, index_file_path)

## Load faiss index and query

In [ ]:
# use this for batch faiss encode (updated to fetch multiple context) + multiple choice added to query vector

faiss_index_path = '/content/drive/MyDrive/faiss_768_break.index'
trans_corpus_path = '/content/drive/MyDrive/df_768_break.csv'
index = faiss.read_index(faiss_index_path)
trans_corpus = pd.read_csv(trans_corpus_path)

def dataset_get_multi_context_batch_with_choices(examples):
    question_batch = examples['trans_ner_question']
    choice1_batch = examples['trans_option_1']
    choice2_batch = examples['trans_option_2']
    choice3_batch = examples['trans_option_3']
    choice4_batch = examples['trans_option_4']
    choice5_batch = examples['trans_option_5']

    choices_batch = [list(item) for item in zip(choice1_batch, choice2_batch, choice3_batch, choice4_batch, choice5_batch)]
    filtered_choices_batch = [[item for item in sublist if item != 'cloudy sky'] for sublist in choices_batch]
    choices_string_batch = [', '.join(item) for item in filtered_choices_batch]

    query_batch = [f'{question}, {choices}' for question, choices in zip(question_batch,choices_string_batch)]

    # batch sentenceTransformers
    model_name = '/content/drive/MyDrive/Sentence_Tramsformers_Model'  #base mp v3
    model = SentenceTransformer(model_name)

    xq_batch = model.encode(query_batch)
    # Create 2 contexts for 1 question
    context_batch_1 = []
    context_batch_2 = []

    for xq in xq_batch:
        xq = np.expand_dims(xq, axis=0)  # query vector
        D, I = index.search(xq, 10)   # D: distance, I: index
        context = [trans_corpus['Content'][i] for i in I[0]]

        ok_context = ''
        ok_batch = []
        for text in context:
            if len(text.split()) <= 300:
                ok_context = ok_context + ' \n' + text
                if len(ok_context.split()) > 350:
                    ok_batch.append(ok_context)
                    ok_context = ''
            else:
                ok_context = ok_context + ' \n' + text
                ok_batch.append(ok_context)
                ok_context = ''
            if len(ok_batch) == 2:
                    break

        context_1 = ok_batch[0]
        context_2 = ok_batch[1]
        context_batch_1.append(context_1)
        context_batch_2.append(context_2)

    examples['context_1'] = context_batch_1
    examples['context_2'] = context_batch_2
    return examples

# Main

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/public_test.csv')

In [ ]:
test_df = test_df.fillna('bầu trời có nhiều mây')

In [ ]:
test_df.head(3)

,id,question,option_1,option_2,option_3,option_4,option_5,option_6
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,bầu trời có nhiều mây,bầu trời có nhiều mây
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,bầu trời có nhiều mây,bầu trời có nhiều mây
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,bầu trời có nhiều mây,bầu trời có nhiều mây


In [ ]:
test_hf = datasets.Dataset.from_pandas(test_df)

# Convert NER Person

In [ ]:
%%time
test_ner = test_hf.map(convert_name_NER_batch, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


CPU times: user 22 s, sys: 428 ms, total: 22.5 s
Wall time: 25.2 s


# Translate dataset

In [ ]:
%%time
test_ner_trans = test_ner.map(translate_dataset_batch, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

CPU times: user 14.4 s, sys: 1.97 s, total: 16.4 s
Wall time: 48.1 s


# Term Mapping

## Prepare term mapping dictionary

In [ ]:
term_df = pd.read_csv('/content/drive/MyDrive/right_term.csv', encoding='utf-8', delimiter =';').drop(columns='Unnamed: 0')
term_df.head(5)

,Url,Name,model_trans_term,gg_trans_term,right_term
0,alzheimer,Bệnh Alzheimer,Alzheimer's disease,Alzheimer,Alzheimer's disease
1,an-khong-tieu,Ăn không tiêu,Indigestion,Eat but can not digest,indigestion
2,ap-xe-hau-mon,Áp xe hậu môn,Anal abscess,Anal abscess,anal abscess
3,ap-xe-phoi,Áp xe phổi,Lung abscess,Lung abscess,lung abscess
4,ban-chan-dai-thao-duong,bàn chân đái tháo đường,diabetic foot,diabetic foot,diabetic foot


In [ ]:
# convert to dict
term_dict = {}
for _, row in term_df.iterrows():
    wrong_term_1 = row['model_trans_term'].lower()
    wrong_term_2 = row['gg_trans_term'].lower()
    target_term = row['right_term'].lower()

    if wrong_term_1 not in term_dict:
        term_dict[wrong_term_1] = target_term
    if wrong_term_2 not in term_dict:
        term_dict[wrong_term_2] = target_term

filtered_term_dict = {k: v for k, v in term_dict.items() if k != v}

In [ ]:
with open('/content/drive/MyDrive/term_dict.json', 'w', encoding ='utf8') as json_file:
    json.dump(term_dict, json_file)

## Load Term Mapping & Apply To Question



In [ ]:
term_dict_path = '/content/drive/MyDrive/term_dict.json'
with open(term_dict_path) as f:
    term_dict = json.load(f)
    f.close()

def to_map(texts):
    mapped_texts = []
    for text in texts:
        for k,v in term_dict.items():
            text = text.lower().replace(k,v)

        mapped_texts.append(text)
    return mapped_texts

def mapping_term_batch(examples):
    examples['trans_ner_question'] = to_map(examples['trans_ner_question'])
    examples['trans_option_1'] = to_map(examples['trans_option_1'])
    examples['trans_option_2'] = to_map(examples['trans_option_2'])
    examples['trans_option_3'] = to_map(examples['trans_option_3'])
    examples['trans_option_4'] = to_map(examples['trans_option_4'])
    examples['trans_option_5'] = to_map(examples['trans_option_5'])

    return examples

In [ ]:
%%time
test_ner_trans_term = test_ner_trans.map(mapping_term_batch, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

CPU times: user 74.3 ms, sys: 0 ns, total: 74.3 ms
Wall time: 73.2 ms


# Get Context For Dataset

In [ ]:
# query context with choice in query
%%time
test_ner_trans_term_context_break_choice= test_ner_trans_term.map(dataset_get_multi_context_batch_with_choices, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

CPU times: user 2.13 s, sys: 259 ms, total: 2.39 s
Wall time: 2.17 s


# Deploy Zero Shot Model

In [ ]:
def create_prompt_zero_shot_model(examples):
    question_batch = examples['trans_ner_question']
    context_batch_1 = examples['context_1']
    context_batch_2 = examples['context_2']

    prompt_batch_1 = [f'Question: {question} [SEP] Answer question given Context: {context}' for question, context in zip(question_batch, context_batch_1)]
    prompt_batch_2 = [f'Question: {question} [SEP] Answer question given Context: {context}' for question, context in zip(question_batch, context_batch_2)]

    return {
            'prompt_1': prompt_batch_1,
            'prompt_2': prompt_batch_2
            }

In [ ]:
%%time
test_ner_trans_term_context_break_prompt_choice = test_ner_trans_term_context_break_choice.map(create_prompt_zero_shot_model,batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

CPU times: user 23.8 ms, sys: 7.2 ms, total: 31 ms
Wall time: 29.3 ms


In [ ]:
def get_question_type(example):
    question = example['trans_ner_question']
    option1 = example['trans_option_1']

    question_kw = ['how early','what stage','what week','how many','in how many','what time']
    choice_kw = ['yes','no','none', 'true', 'false']

    n_choice = 'multi'
    for kw in question_kw:
        if re.search(r'\b' + re.escape(kw) + r'\b', question):
            n_choice = 'single'

    for kw in choice_kw:
        if re.search(r'\b' + re.escape(kw) + r'\b', option1):
            n_choice = 'single'

    example['n_choice'] = n_choice
    return example

In [ ]:
test_ner_trans_term_context_break_prompt_choice.save_to_disk('/content/drive/MyDrive/test_ner_trans_term_context_break_prompt_choice.hf')

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
test_ner_trans_term_context_break_prompt_choice = datasets.load_from_disk('/content/drive/MyDrive/test_ner_trans_term_context_break_prompt_choice.hf')

In [ ]:
test_ner_trans_term_context_break_prompt_choice = test_ner_trans_term_context_break_prompt_choice.select_columns(['id','trans_ner_question','trans_option_1',
                                                                            'trans_option_2','trans_option_3','trans_option_4',
                                                                            'trans_option_5','context_1','context_2','prompt_1','prompt_2'])

In [ ]:
test_ner_trans_term_context_break_prompt_choice = test_ner_trans_term_context_break_prompt_choice.map(get_question_type)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
pd.DataFrame(test_ner_trans_term_context_break_prompt_choice).head(3)

,id,trans_ner_question,trans_option_1,trans_option_2,trans_option_3,trans_option_4,trans_option_5,context_1,context_2,prompt_1,prompt_2,n_choice
0,level3_1,person1 is pregnant and is worried about the p...,week 10,week 20,30th,week 40,cloudy sky,\nHow many weeks will it take to find out pla...,\nDiagnostic method. Ultrasound methods are a...,Question: person1 is pregnant and is worried a...,Question: person1 is pregnant and is worried a...,single
1,level3_2,person1 is in her fifth week of pregnancy and ...,5 weeks,15 weeks,25 weeks,35 weeks,cloudy sky,\nHow many weeks will it take to find out pla...,\nDiagnostic method. Creatinine is a waste pr...,Question: person1 is in her fifth week of preg...,Question: person1 is in her fifth week of preg...,single
2,level3_5,how many of these are prolactin placenta praev...,2,3,4,5,cloudy sky,\nWhat is placenta praevia? Placenta previa i...,\nTypes of pelvic organ prolapse. Bladder pro...,Question: how many of these are prolactin plac...,Question: how many of these are prolactin plac...,single


In [ ]:
torch.cuda.empty_cache()

# Ensemble Method

In [ ]:
%%time
classifier1 = pipeline("zero-shot-classification", model="/content/drive/MyDrive/Zero Shot Model/bart-large-mnli", device=0, use_fast=True)
classifier2 = pipeline("zero-shot-classification", model="/content/drive/MyDrive/Zero Shot Model/DeBERTa-v3-base-mnli-fever-anli", device=0, use_fast=True)
classifier3 = pipeline("zero-shot-classification", model="/content/drive/MyDrive/Zero Shot Model/Scandi-nli-large", device=0, use_fast=True)
classifier4 = pipeline("zero-shot-classification", model="/content/drive/MyDrive/Zero Shot Model/distilcamembert-base-nli", device=0, use_fast=True)
classifier5 = pipeline("zero-shot-classification", model="/content/drive/MyDrive/Zero Shot Model/xlm-roberta-large-xnli-anli", device=0, use_fast=True)
classifier6 = pipeline("zero-shot-classification", model="/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9", device=0, use_fast=True)
classifier7 = pipeline("zero-shot-classification", model="/content/drive/MyDrive/Zero Shot Model/Ndugar-3poch-3large", device=0, use_fast=True)

classifiers = [classifier1, classifier2, classifier3, classifier4, classifier5, classifier6, classifier7]

CPU times: user 30.7 s, sys: 7.17 s, total: 37.8 s
Wall time: 4min 3s


In [ ]:
#4132124, 10, 14

In [ ]:
# Weighted Vote
def get_binary_answer(choices: list, answers: list):
    length = len(choices)
    result = ['0']*length
    for answer in answers:
        for i in range(len(choices)):
            if answer == choices[i]:
                result[i] = '1'
    return ''.join(result)

def weight_ensemble_binary(scores, weight_dist=[4, 1, 3, 2, 1, 2, 4], vote=10):
    length = len(scores[0])
    votes = []
    answer = ['0'] * length
    for i in range(length):
        count = 0
        for c in range(len(scores)):
            if scores[c][i] == '1':
                count += weight_dist[c]
        if count >= vote:
            answer[i] = '1'
            votes.append((i, count))  # Store the index and count in a tuple

    return ''.join(answer), votes

def final_ensemble(scores):
    length = len(scores[0])
    answer = ['0']*length
    for i in range(len(scores[0])):
        for j in range(len(scores)):
          if scores[j][i] == '1':
            answer[i] = '1'
            break
    return ''.join(answer)

def get_multi_context_prediction_ensemble_weight_vote(example):
    prompt_1 = example['prompt_1']
    prompt_2 = example['prompt_2']
    id = example['id']
    n_choice = example['n_choice']
    choices = [example['trans_option_1'], example['trans_option_2'],
               example['trans_option_3'], example['trans_option_4'], example['trans_option_5']]

    choices = [item for item in choices if item != 'cloudy sky']
    length = len(choices)

    threshold = 0.8
    argmax_threshold1 = 0.65
    argmax_threshold2 = 0.75
    context1_for_ensemble = [] # result will be as binary : ['1001', '1001','1000',.....,'0001'] then used to merge into final answer
    context2_for_ensemble = []

    for classifier in classifiers:
        results = classifier([prompt_1, prompt_2], choices, multi_label=True)
        context1_scores, context1_labels = results[0]['scores'], results[0]['labels']
        context2_scores, context2_labels = results[1]['scores'], results[1]['labels']

        candidate1_answer = []
        candidate2_answer = []
        if np.max(context1_scores) >= argmax_threshold1: # make sure max score > threshold
            max_score_1_index = np.argmax(context1_scores)
            candidate1_answer.append(context1_labels[max_score_1_index])

        if np.max(context2_scores) >= argmax_threshold2: # make sure max score > threshold
            max_score_2_index = np.argmax(context2_scores)
            candidate2_answer.append(context2_labels[max_score_2_index])

        # Include other labels with scores above the threshold with choice = 'multi'
        if n_choice == 'multi':
            for i in range(len(choices)):
                if (context1_scores[i] >= threshold) and (context1_labels[i] not in candidate1_answer):
                    candidate1_answer.append(context1_labels[i])

                if (context2_scores[i] >= threshold) and (context2_labels[i] not in candidate2_answer):
                    candidate2_answer.append(context2_labels[i])

        bin_candidate1_answer = get_binary_answer(choices, candidate1_answer)
        bin_candidate2_answer = get_binary_answer(choices, candidate2_answer)

        context1_for_ensemble.append(bin_candidate1_answer)
        context2_for_ensemble.append(bin_candidate2_answer)

    # Begin to ensemble answer
    context1_final_answer, context1_votes_list = weight_ensemble_binary(context1_for_ensemble, weight_dist=[4, 1, 3, 2, 1, 2, 4], vote=10)
    context2_final_answer, context2_votes_list = weight_ensemble_binary(context2_for_ensemble, weight_dist=[3, 2, 3, 4, 3, 3, 2], vote=15)
    if context1_votes_list:
        context1_votes = context1_votes_list[0][1]
    else:
        context1_votes = 0

    if context2_votes_list:
        context2_votes = context2_votes_list[0][1]
    else:
        context2_votes = 0

    # Final ensemble
    if n_choice == 'multi':  # '1001' + '1100' = '1101'
        final_answer = final_ensemble([context1_final_answer, context2_final_answer])

    if n_choice == 'single': # '1000' + '0001' = higher vote of 2
        if context1_final_answer == context2_final_answer:
            final_answer = context1_final_answer
        else:
            if context1_votes >= context2_votes:
                final_answer = context1_final_answer
            else:
                final_answer = context2_final_answer

    null_answer_list = ['0']*length
    null_answer = ''.join(null_answer_list)
    if final_answer == null_answer:   # Đánh lụi
        null_answer_list[1] = '1'
        final_answer = ''.join(null_answer_list)

    binary_final_answer = f'{id},{final_answer}'

    example['context1_for_ensemble'] = context1_for_ensemble
    example['context2_for_ensemble'] = context2_for_ensemble
    example['context1_answer'] = context1_final_answer
    example['context2_answer'] = context2_final_answer
    example['answer'] = final_answer
    # example['id,answer'] = binary_final_answer

    return example

In [ ]:
%%time
prediction_weighted_vote_break_choice = test_ner_trans_term_context_break_prompt_choice.map(get_multi_context_prediction_ensemble_weight_vote)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


CPU times: user 9min 7s, sys: 5.77 s, total: 9min 13s
Wall time: 9min 10s


In [ ]:
prediction_break = pd.DataFrame(prediction_weighted_vote_break_choice)

In [ ]:
prediction_break.sample(5)

,id,trans_ner_question,trans_option_1,trans_option_2,trans_option_3,trans_option_4,trans_option_5,context_1,context_2,prompt_1,prompt_2,n_choice,context1_for_ensemble,context2_for_ensemble,context1_answer,context2_answer,answer
58,level1_106,how to prevent laryngitis.,avoid smoking and smoking cessation,drink plenty of water,avoid late eating,cloudy sky,cloudy sky,\nComplications and transmission routes. Note...,\nSymptoms of laryngitis. Difficulty swallowi...,Question: how to prevent laryngitis. [SEP] Ans...,Question: how to prevent laryngitis. [SEP] Ans...,multi,"[111, 011, 111, 111, 010, 111, 011]","[100, 000, 001, 111, 100, 000, 000]",111,000,111
75,level2_12,what is the diagnostic method for chlamydia an...,blood tests,urine test,vaginal fluid test,cloudy sky,cloudy sky,\nMethod for diagnosing persistent vaginitis....,\nDisease diagnostic tests. Blood test: Sexua...,Question: what is the diagnostic method for ch...,Question: what is the diagnostic method for ch...,multi,"[001, 001, 010, 111, 011, 001, 001]","[111, 100, 100, 111, 111, 101, 101]",001,101,101
11,level3_55,the doctor diagnoses person1 as brain tumor. a...,yes,no,unknown,cloudy sky,cloudy sky,"\nIs brain tumor dangerous? Therefore, the ea...",\nClassification of brain tumors. Characteris...,Question: the doctor diagnoses person1 as brai...,Question: the doctor diagnoses person1 as brai...,single,"[000, 100, 000, 001, 100, 000, 000]","[000, 000, 000, 001, 000, 000, 000]",000,000,010
62,level1_40,what should one do to prevent bursitis?,practice good posture in daily life and work,avoid over-exertion,frequent or repetitive knee slumping can put a...,cloudy sky,cloudy sky,\nPrevention of knee bursitis. People with kn...,\nTreatments. When joint and bursa damage can...,Question: what should one do to prevent bursit...,Question: what should one do to prevent bursit...,multi,"[111, 111, 011, 111, 011, 111, 111]","[110, 010, 111, 111, 111, 111, 111]",111,111,111
13,level3_93,the urethra is a fairly important genital orga...,urinate several times a day,urine with pus or fresh blood,"the urinal is red and swollen, itchy",cloudy sky,cloudy sky,\nSymptoms of urinary tract infection. Patien...,\nSymptoms and signs of cystitis. Symptoms of...,Question: the urethra is a fairly important ge...,Question: the urethra is a fairly important ge...,multi,"[110, 000, 000, 111, 000, 010, 110]","[100, 110, 010, 111, 111, 110, 110]",110,110,110


In [ ]:
prediction_break[['id','answer']].to_csv('submission35.csv', index=False)

# Evaluation

## Process Answer Key File

In [ ]:
answer_df = pd.read_excel('/content/drive/MyDrive/Answer_Keys.xlsx')
answer_df['Answer_Keys'] = answer_df['Answer_Keys'].astype(str)

In [ ]:
def get_number_choices(row):
    option_1 = row['option_1']
    option_2 = row['option_2']
    option_3 = row['option_3']
    option_4 = row['option_4']
    option_5 = row['option_5']

    if not pd.isnull(option_5):
        choices = 5
    elif not pd.isnull(option_4):
        choices = 4
    elif not pd.isnull(option_3) :
        choices = 3
    else:
        choices = 2
    return choices

answer_df['choices'] = answer_df.apply(get_number_choices, axis=1)
answer_df = answer_df[['id','Answer_Keys','choices']]

In [ ]:
def format_answer_key(row):
    answer = row['Answer_Keys']
    choices = row['choices']
    length = len(answer)
    padding = choices - length

    answer_formated = '0'*padding + answer
    return answer_formated

answer_df['Answer_Keys_Formated'] = answer_df.apply(format_answer_key, axis=1)

In [ ]:
prediction_weighted_vote_df = prediction_weighted_vote_df.merge(answer_df,on='id',how='left')

In [ ]:
prediction_weighted_vote_df

,id,trans_ner_question,trans_option_1,trans_option_2,trans_option_3,trans_option_4,trans_option_5,context_1,context_2,prompt_1,...,context2_for_ensemble,context1_answer,context2_answer,answer,Answer_Keys_x,choices_x,Answer_Keys_Formated_x,Answer_Keys_y,choices_y,Answer_Keys_Formated_y
0,level3_1,person1 is pregnant and is worried about the p...,week 10,week 20,30th,week 40,cloudy sky,\nWith the development of modern medicine and...,\nMost cases of placenta previa can be detect...,Question: person1 is pregnant and is worried a...,...,"[0100, 0100, 0100, 0000, 0100, 0100]",0000,0100,0100,100,4,0100,100,4,0100
1,level3_2,person1 is in her fifth week of pregnancy and ...,5 weeks,15 weeks,25 weeks,35 weeks,cloudy sky,\nPregnant women may have preeclampsia with s...,"\nNormally, 4 to 6 weeks after giving birth, ...",Question: person1 is in her fifth week of preg...,...,"[1000, 0000, 1000, 0001, 0100, 1000]",1000,1000,1000,100,4,0100,100,4,0100
2,level3_5,how many of these are prolactin placenta praev...,2,3,4,5,cloudy sky,\nPlacenta previa is a phenomenon in which th...,"\nWhen the cervix is open, the edge of the pl...",Question: how many of these are prolactin plac...,...,"[0100, 0100, 0100, 0010, 0000, 0100]",1000,0100,1100,1,4,0001,1,4,0001
3,level3_13,a patient with person1 was diagnosed with hepa...,yes,no,cloudy sky,cloudy sky,cloudy sky,\nChronic hepatitis is liver damage that last...,\nChronic hepatitis due to non-alcoholic fatt...,Question: a patient with person1 was diagnosed...,...,"[00, 10, 10, 10, 01, 00]",10,10,10,1,2,01,1,2,01
4,level3_14,a patient presented with testicular pain. afte...,urinary tract infection,testicular cancer,injury,varicocele,cloudy sky,\nRight testicular pain is more likely to occ...,\nTesticular pain is a painful condition that...,Question: a patient presented with testicular ...,...,"[1011, 0010, 0000, 1111, 0010, 0011]",1111,1011,1111,1,4,0001,1,4,0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,level4_4,person1 sugar is a famous female singer in chi...,tonsillitis - related pharyngitis,diabetes mellitus,nasopharyngeal carcinoma,internal hemorrhoids,cloudy sky,\nVocal abuse can cause the vocal cords to st...,"\nAllergies, colds or upper respiratory infec...",Question: person1 sugar is a famous female sin...,...,"[0000, 0010, 0010, 0001, 1000, 0000]",1001,0000,1001,1010,4,1010,1010,4,1010
102,level4_9,the morning is now 9 months pregnant. there ar...,genital herpes,gonorrhea,chlamydia,syphilis,cloudy sky,\nFull peritonitis can occur about 7 - 10 day...,\nPregnant women with sepsis are treated with...,Question: the morning is now 9 months pregnant...,...,"[0000, 0000, 0000, 0001, 0000, 0000]",0000,0000,0000,111,4,0111,111,4,0111
103,level4_27,mr. person1 is 73 years old. at a family dinne...,brain tumor,brain stem deficiency,cerebral embolism,stroke,cloudy sky,"\nFacial paralysis, also known as 7th nerve p...",\nGastroesophageal reflux disease: The esopha...,Question: mr. person1 is 73 years old. at a fa...,...,"[0000, 0000, 0000, 1111, 0000, 1110]",0100,0000,0100,11,4,0011,11,4,0011
104,level4_28,brain tumor are neoplasms that form in the cra...,they're all serioudengue hemorrhagic fevers.,it's all cancer.,exact cause unknown,happens most often in old people.,cloudy sky,\nA brain tumor is a collection of large numb...,"\nBenign brain tumors are not cancerous, they...",Question: brain tumor are neoplasms that form ...,...,"[0001, 0011, 0010, 0000, 0010, 0011]",0000,0011,0011,1011,4,1011,1011,4,1011


## Download Model to Disk to run offline

In [ ]:
!pip install sentencepiece
!pip install deepspeed

  Created wheel for deepspeed: filename=deepspeed-0.12.3-py3-none-any.whl size=1279166 sha256=9fd15a3406448fa118f813732d897d23f201c89c9fc1223ba89937d376281121
  Stored in directory: /root/.cache/pip/wheels/ee/2b/c5/892ceee06964ce8aa2a98d4260848d0d9a3f1e743862e4b45a
Successfully built deepspeed


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Replace with your desired model and tokenizer
model_name = "NDugar/3epoch-3large"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Specify local directory to save the model and tokenizer
local_directory = "/content/drive/MyDrive/Zero Shot Model/Ndugar-3poch-3large"

# Save the model and tokenizer to the specified directory
model.save_pretrained(local_directory)
tokenizer.save_pretrained(local_directory)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


('/content/drive/MyDrive/Zero Shot Model/Ndugar-3poch-3large/tokenizer_config.json',
 '/content/drive/MyDrive/Zero Shot Model/Ndugar-3poch-3large/special_tokens_map.json',
 '/content/drive/MyDrive/Zero Shot Model/Ndugar-3poch-3large/spm.model',
 '/content/drive/MyDrive/Zero Shot Model/Ndugar-3poch-3large/added_tokens.json',
 '/content/drive/MyDrive/Zero Shot Model/Ndugar-3poch-3large/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import sentencepiece

model7 = AutoModelForSequenceClassification.from_pretrained("valhalla/distilbart-mnli-12-9")
tokenizer7 = AutoTokenizer.from_pretrained('valhalla/distilbart-mnli-12-9',use_fast=True)

model7.save_pretrained("/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9")
tokenizer7.save_pretrained("/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9")

('/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9/tokenizer_config.json',
 '/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9/special_tokens_map.json',
 '/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9/vocab.json',
 '/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9/merges.txt',
 '/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9/added_tokens.json',
 '/content/drive/MyDrive/Zero Shot Model/distilbart-mnli-12-9/tokenizer.json')

In [ ]:
prediction_df = pd.DataFrame(prediction)

NameError: ignored

In [ ]:
prediction_df.sample(5)

,id,question,option_1,option_2,option_3,option_4,option_5,option_6,ner_question,trans_ner_question,...,trans_option_3,trans_option_4,trans_option_5,context_1,context_2,prompt_1,prompt_2,context1_answer,context2_answer,binary_final_answer
11,level3_55,Bác sĩ chuẩn đoán Lan bị mắc bệnh u não. Sau k...,A. Có,B. Không,C. Không rõ,bầu trời có nhiều mây,bầu trời có nhiều mây,bầu trời có nhiều mây,Bác sĩ chuẩn đoán Person1 bị mắc bệnh u não. S...,The doctor diagnoses Person1 as brain tumor. A...,...,Unknown,Cloudy sky,Cloudy sky,\nBrain tumors in different brain regions wil...,\nThe process of diagnosing benign brain tumo...,Question: The doctor diagnoses Person1 as brai...,Question: The doctor diagnoses Person1 as brai...,000,101,"level3_55,101"
44,level1_54,Các biến chứng của suy tim phải bao gồm những ...,A. Rung nhĩ,B.Suy tim,C. Các vấn đề về van tim,D. Suy giảm chức năng thận,E. Tổn thương gan,bầu trời có nhiều mây,Các biến chứng của suy tim phải bao gồm những ...,What should complications of heart failure inc...,...,Valvular problems,Impaired kidney function,Liver damage,\nComplications become more severe in the lat...,"\nBefore a patient with heart failure, it is ...",Question: What should complications of heart f...,Question: What should complications of heart f...,11111,01111,"level1_54,11111"
20,level1_38,Bác sĩ có thể chỉ định người bệnh thực hiện mộ...,A. Chụp X-quang,B.Chụp MRI,C. Siêu âm,bầu trời có nhiều mây,bầu trời có nhiều mây,bầu trời có nhiều mây,Bác sĩ có thể chỉ định người bệnh thực hiện mộ...,Your doctor may order imaging tests such as:?,...,Ultrasound,Cloudy sky,Cloudy sky,"\nFirst, the doctor will examine and ask abou...",\nParaclinical examinations such as diagnosti...,Question: Your doctor may order imaging tests ...,Question: Your doctor may order imaging tests ...,001,100,"level1_38,101"
58,level1_106,Làm thế nào để phòng ngừa bệnh viêm thanh quản?.,A. Tránh hút thuốc và tránh xa khói thuốc,B.Uống nhiều nước,C. Tránh ăn khuya,bầu trời có nhiều mây,bầu trời có nhiều mây,bầu trời có nhiều mây,Làm thế nào để phòng ngừa bệnh viêm thanh quản?.,How to prevent laryngitis.,...,Avoid Late Eating,Cloudy sky,Cloudy sky,"\nTo prevent laryngitis, you need to follow t...",\nLaryngeal cancer is a dangerous disease but...,Question: How to prevent laryngitis. [SEP] Ans...,Question: How to prevent laryngitis. [SEP] Ans...,111,111,"level1_106,111"
92,level2_38,"Triệu chứng chung của suy tim, suy hô hấp, suy...",Sưng chân và mắt cá chân,"Ngón tay, ngón chân và môi xanh xao, nhợt nhạt",Dạ dày,Khó thở,bầu trời có nhiều mây,bầu trời có nhiều mây,"Triệu chứng chung của suy tim, suy hô hấp, suy...","General symptoms of heart failure, respiratory...",...,Stomach,Shortness of breath,Cloudy sky,\nSymptoms of the disease may be different fo...,\nHeart failure is one of the common causes o...,"Question: General symptoms of heart failure, r...","Question: General symptoms of heart failure, r...",1001,1011,"level2_38,1011"


# Testing different Classifier

In [ ]:
# function for multi context
import time

def get_prompt_answer_multi_context(df, index):
    row = df.iloc[index]
    prompt1 = row['prompt_1']
    prompt2 = row['prompt_2']
    option1 = row['trans_option_1']
    option2 = row['trans_option_2']
    option3 = row['trans_option_3']
    option4 = row['trans_option_4']
    option5 = row['trans_option_5']
    choices = [option1, option2, option3, option4, option5]
    choices = [item for item in choices if item != 'cloudy sky']
    prompts = [prompt1, prompt2]
    id = row['id']
    return id, prompts, choices

def print_result_testing():
    start_time = time.time()
    results1 = classifier1(prompts, choices, multi_label=True)
    end_time = time.time()  # End time after the classifier executions
    print(f"Time taken by classifier 1: {end_time - start_time} seconds")

    start_time = time.time()
    results2 = classifier2(prompts, choices, multi_label=True)
    end_time = time.time()  # End time after the classifier executions
    print(f"Time taken by classifier 2: {end_time - start_time} seconds")

    start_time = time.time()
    results3 = classifier3(prompts, choices, multi_label=True)
    end_time = time.time()  # End time after the classifier executions
    print(f"Time taken by classifier 3: {end_time - start_time} seconds")

    start_time = time.time()
    results4 = classifier4(prompts, choices, multi_label=True)
    end_time = time.time()  # End time after the classifier executions
    print(f"Time taken by classifier 4: {end_time - start_time} seconds")

    start_time = time.time()
    results5 = classifier5(prompts, choices, multi_label=True)
    end_time = time.time()  # End time after the classifier executions
    print(f"Time taken by classifier 5: {end_time - start_time} seconds")

    start_time = time.time()
    results6 = classifier6(prompts, choices, multi_label=True)
    end_time = time.time()  # End time after the classifier executions
    print(f"Time taken by classifier 6: {end_time - start_time} seconds")

    print(id)
    print(results1[0]['sequence'])
    print('----------------------')
    print(results1[1]['sequence'])
    print('----------------------')

    print('######## Classifier 1')
    print(results1[0]['labels'])
    print(results1[0]['scores'])
    print(results1[1]['labels'])
    print(results1[1]['scores'])
    print('######## Classifier 2')
    print(results2[0]['labels'])
    print(results2[0]['scores'])
    print(results2[1]['labels'])
    print(results2[1]['scores'])
    print('######## Classifier 3')
    print(results3[0]['labels'])
    print(results3[0]['scores'])
    print(results3[1]['labels'])
    print(results3[1]['scores'])
    print('######## Classifier 4')
    print(results4[0]['labels'])
    print(results4[0]['scores'])
    print(results4[1]['labels'])
    print(results4[1]['scores'])
    print('######## Classifier 5')
    print(results5[0]['labels'])
    print(results5[0]['scores'])
    print(results5[1]['labels'])
    print(results5[1]['scores'])
    print('######## Classifier 6')
    print(results6[0]['labels'])
    print(results6[0]['scores'])
    print(results6[1]['labels'])
    print(results6[1]['scores'])
    print('########')

id, prompts, choices = get_prompt_answer_multi_context(prediction_df, 72)
print_result_testing()

Time taken by classifier 1: 1.2345411777496338 seconds
Time taken by classifier 2: 0.32766246795654297 seconds
Time taken by classifier 3: 0.6425495147705078 seconds
Time taken by classifier 4: 0.11101388931274414 seconds
Time taken by classifier 5: 0.644711971282959 seconds
Time taken by classifier 6: 1.0199482440948486 seconds
level2_3
Question: how do you differentiate acute and chronic pharyngitis? [SEP] Answer question given Context:  
Throat culture is the "gold standard" in diagnosing acute pharyngitis. Parainfluenza virus infection can cause bronchiolitis; mild sore throat but quickly resolved. Antibiotic treatment reduces the incidence of sequelae, such as retropharyngeal abscesses. When an abscess is detected clinically or by imaging diagnostics such as computed tomography (CT), surgical drainage will be indicated. The antibiotics Penicillin or Amoxicillin are indicated for patients with a positive RADT or throat culture. According to  Huong, depending on the level of inflamm

In [ ]:
# nếu score của classifier1 ko có score nào cao thì sẽ retrieve đoạn text khác

# Testing

In [ ]:
# Weighted Vote with confident
def get_binary_answer(choices: list, answers: list):
    length = len(choices)
    result = ['0']*length
    for answer in answers:
        for i in range(len(choices)):
            if answer == choices[i]:
                result[i] = '1'
    return ''.join(result)

def weight_ensemble_binary(scores, weight_dist, vote): # take list of scores and return 1 string of legitimate votes and a list record vote '1'
    length = len(scores[0])
    votes = []
    answer = ['0'] * length
    for i in range(length):
        count = 0
        for c in range(len(scores)):
            if scores[c][i] == '1':
                count += weight_dist[c]
        if count >= vote:
            answer[i] = '1'
            votes.append((i, count))  # Store the index and count in a tuple
    votes.sort(key=lambda x: x[1], reverse=True)
    return ''.join(answer), votes

def final_ensemble(scores):
    length = len(scores[0])
    answer = ['0']*length
    for i in range(len(scores[0])):
        for j in range(len(scores)):
          if scores[j][i] == '1':
            answer[i] = '1'
            break
    return ''.join(answer)

def get_multi_context_prediction_ensemble_weight_vote(example):
    prompt_1 = example['prompt_1']
    prompt_2 = example['prompt_2']
    id = example['id']
    n_choice = example['n_choice']
    choices = [example['trans_option_1'], example['trans_option_2'],
               example['trans_option_3'], example['trans_option_4'], example['trans_option_5']]

    choices = [item for item in choices if item != 'cloudy sky']
    length = len(choices)

    confident_threshold = 0.98
    threshold = 0.8
    confident_answers = []
    argmax_threshold1 = 0.65
    argmax_threshold2 = 0.75
    context1_for_ensemble = [] # result will be as binary : ['1001', '1001','1000',.....,'0001'] then used to merge into final answer
    context2_for_ensemble = []


    for classifier in classifiers:
        results = classifier([prompt_1, prompt_2], choices, multi_label=True)
        context1_scores, context1_labels = results[0]['scores'], results[0]['labels']
        context2_scores, context2_labels = results[1]['scores'], results[1]['labels']

        candidate1_answer = []
        candidate2_answer = []

        # Code to add confident answer in binary and append to list of confident_answers
        confident_labels = set()
        for score, label in zip(context1_scores,context1_labels):
            if score >= confident_threshold:
                confident_labels.add(label)

        for score, label in zip(context2_scores,context2_labels):
            if score >= confident_threshold:
                confident_labels.add(label)

        confident_answer = get_binary_answer(choices, confident_labels) # Get answer of confident labels
        confident_answers.append(confident_answer)

        if np.max(context1_scores) >= argmax_threshold1: # make sure max score > threshold
            max_score_1_index = np.argmax(context1_scores)
            candidate1_answer.append(context1_labels[max_score_1_index])

        if np.max(context2_scores) >= argmax_threshold2: # make sure max score > threshold
            max_score_2_index = np.argmax(context2_scores)
            candidate2_answer.append(context2_labels[max_score_2_index])

        # Include other labels with scores above the threshold with choice = 'multi'
        if n_choice == 'multi':
            for i in range(len(choices)):
                if (context1_scores[i] >= threshold) and (context1_labels[i] not in candidate1_answer):
                    candidate1_answer.append(context1_labels[i])

                if (context2_scores[i] >= threshold) and (context2_labels[i] not in candidate2_answer):
                    candidate2_answer.append(context2_labels[i])

        bin_candidate1_answer = get_binary_answer(choices, candidate1_answer)
        bin_candidate2_answer = get_binary_answer(choices, candidate2_answer)

        context1_for_ensemble.append(bin_candidate1_answer)
        context2_for_ensemble.append(bin_candidate2_answer)

    # Begin to ensemble answer
    context1_final_answer, context1_votes_list = weight_ensemble_binary(context1_for_ensemble, weight_dist=[4, 1, 3, 2, 1, 2, 4], vote=10)
    context2_final_answer, context2_votes_list = weight_ensemble_binary(context2_for_ensemble, weight_dist=[3, 2, 3, 4, 3, 3, 2], vote=15)
    confident_final_answer, confident_votes_list = weight_ensemble_binary(confident_answers, weight_dist=[2, 1, 1, 1, 1, 1, 1], vote=4)
    print('confident answer', confident_final_answer)
    if context1_votes_list:
        context1_votes = context1_votes_list[0][1]
    else:
        context1_votes = 0

    if context2_votes_list:
        context2_votes = context2_votes_list[0][1]
    else:
        context2_votes = 0

    # Final ensemble
    if n_choice == 'multi':  # '1001' + '1100' = '1101'
        final_answer = final_ensemble([context1_final_answer, context2_final_answer, confident_final_answer])

    if n_choice == 'single': # '1000' + '0001' = higher vote of 2
        if context1_final_answer == context2_final_answer:
            final_answer = context1_final_answer
        else:
            if context1_votes >= context2_votes:
                final_answer = context1_final_answer
            else:
                final_answer = context2_final_answer

    # when answer is all '00000'
    null_answer_list = ['0']*length
    null_answer = ''.join(null_answer_list)
    if (final_answer == null_answer) and (confident_final_answer != null_answer) and (n_choice == 'multi'):
        final_answer = confident_final_answer
    elif (final_answer == null_answer) and (confident_final_answer != null_answer) and (n_choice == 'single'): # confident '1001' -> '1000'
        for i in range(len(confident_final_answer)):
            if confident_final_answer[i] == '1':
                index_1 = i
                print('index lui', index_1)
                break
        null_answer_list[index_1] = '1'   # Take confident answer as final
        final_answer = ''.join(null_answer_list)
    elif (final_answer == null_answer) and (confident_final_answer == null_answer):
        null_answer_list[1] = '1'  # Danh Lui
        final_answer = ''.join(null_answer_list)
    print(final_answer)

    binary_final_answer = f'{id},{final_answer}'
    example['context1_for_ensemble'] = context1_for_ensemble
    example['context2_for_ensemble'] = context2_for_ensemble
    example['context1_answer'] = context1_final_answer
    example['context2_answer'] = context2_final_answer
    example['answer'] = final_answer
    example['id,answer'] = binary_final_answer

    return example

In [ ]:
%%time
prediction_weighted_vote_break_choice = test_ner_trans_term_context_break_prompt_choice.map(get_multi_context_prediction_ensemble_weight_vote)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


confident answer 0000
0100
confident answer 1000
index lui 0
1000
confident answer 0000
0100
confident answer 00
10
confident answer 0000
0010
confident answer 0000
1000
confident answer 0000
0100
confident answer 00
01
confident answer 000
101
confident answer 00
01
confident answer 00000
11111
confident answer 000
010
confident answer 000
010
confident answer 000
110
confident answer 000
100
confident answer 110
111
confident answer 0000
0100
confident answer 1000
1001
confident answer 100
100
confident answer 000
010
confident answer 000
100
confident answer 001
011
confident answer 000
111
confident answer 000
111
confident answer 0000
0100
confident answer 00
10
confident answer 010
010
confident answer 000
110
confident answer 0000
1100
confident answer 0101
0101
confident answer 0000
0101
confident answer 0000
0100
confident answer 000
111
confident answer 000
100
confident answer 001
001
confident answer 100
111
confident answer 0000
0100
confident answer 00
11
confident answer

In [ ]:
prediction_break = pd.DataFrame(prediction_weighted_vote_break_choice)

In [ ]:
prediction_break.sample(20)

,id,trans_ner_question,trans_option_1,trans_option_2,trans_option_3,trans_option_4,trans_option_5,context_1,context_2,prompt_1,prompt_2,n_choice,context1_for_ensemble,context2_for_ensemble,context1_answer,context2_answer,answer,"id,answer"
32,level1_91,what can urethritis do?,prostatitis,epididymitis,infertility,cloudy sky,cloudy sky,\nComplications of urethritis. Bacterial uret...,\nCause of epididymitis. Non-infectious epidi...,Question: what can urethritis do? [SEP] Answer...,Question: what can urethritis do? [SEP] Answer...,multi,"[011, 110, 111, 111, 000, 011, 111]","[110, 110, 110, 000, 111, 010, 010]",111,010,111,"level1_91,111"
72,level2_3,how do you differentiate acute and chronic pha...,"over a prolonged period of time, chronic phary...",by differentiating between the causes of the d...,distinguish between acute throat infection and...,cloudy sky,cloudy sky,\nWhat is acute pharyngitis? Acute pharyngiti...,\nWhat is sore throat? Pharyngitis is a condi...,Question: how do you differentiate acute and c...,Question: how do you differentiate acute and c...,multi,"[011, 010, 011, 111, 111, 010, 011]","[110, 111, 111, 111, 011, 010, 110]",011,010,011,"level2_3,011"
12,level3_76,person1 is a famous model in vietnam. for a mo...,get 4 hours of sleep a day,eat more sweets,drink 1.5 – 2 liters (0.6 – 0.9 us gal) of wat...,cloudy sky,cloudy sky,\nNotes to know after treating pitted scars. ...,\nCare to protect skin from hormonal acne. Wa...,Question: person1 is a famous model in vietnam...,Question: person1 is a famous model in vietnam...,multi,"[000, 101, 000, 111, 000, 000, 001]","[101, 001, 001, 001, 000, 100, 101]",000,000,010,"level3_76,010"
33,level1_107,what causes acute laryngitis?,viral infections,smoking,eat spicy and hot food,cloudy sky,cloudy sky,\nWhat is laryngitis? Laryngitis is inflammat...,\nSymptoms of laryngitis. Difficulty swallowi...,Question: what causes acute laryngitis? [SEP] ...,Question: what causes acute laryngitis? [SEP] ...,multi,"[000, 100, 010, 111, 111, 000, 100]","[100, 100, 110, 111, 100, 100, 000]",000,100,100,"level1_107,100"
91,level2_37,"food poisoning, stomach pain, gastrointestinal...","eat it ripe, drink it boiled","dirty, rotten food",stressed,exercise regularly,cloudy sky,\nSymptoms of being hit by food. Poisoning by...,\nSubjects prone to digestive disorders. Peop...,"Question: food poisoning, stomach pain, gastro...","Question: food poisoning, stomach pain, gastro...",multi,"[0010, 0110, 0010, 1111, 0010, 0010, 0110]","[0010, 0110, 0010, 1111, 1010, 0010, 0110]",0010,0010,0010,"level2_37,0010"
65,level1_10,what's the symptoms of hepatitis?,"headache, muscle pain, joint obesityigue","jaundice, yellow eyes","tired, anorexic",cloudy sky,cloudy sky,\nSymptoms of chronic hepatitis. The spleen i...,\nWhat is hepatitis? Hepatitis occurs when li...,Question: what's the symptoms of hepatitis? [S...,Question: what's the symptoms of hepatitis? [S...,multi,"[010, 010, 010, 111, 000, 010, 010]","[111, 011, 011, 111, 001, 010, 011]",010,011,011,"level1_10,011"
44,level1_54,what should complications of heart failure inc...,atrial fibrillation,heart failure,valvular problems,impaired kidney function,liver damage,\nIs congestive heart failure dangerous? When...,\nWhat is heart failure? Heart failure is a c...,Question: what should complications of heart f...,Question: what should complications of heart f...,multi,"[00000, 11100, 00001, 11111, 11110, 00000, 01000]","[01100, 01000, 00100, 11111, 11100, 01100, 01100]",00000,01100,01100,"level1_54,01100"
45,level1_73,why can acne affect your psychology?,acne only affects the appearance,acne does not affect psychology.,acne can make people lose confidence,cloudy sky,cloudy sky,\nWhat is hormonal acne? Acne can affect your...,\nSymptoms of acne formation. Whiteheads are ...,Question: why can acne affect your psychology?...,Question: why can acne affect your psychology?...,multi,"[001, 001, 001, 111, 001, 001, 001]","[000, 001, 001, 111, 001, 001, 001]",001,

In [ ]:
prediction_break[['id','answer']].to_csv('submission37.csv', index=False)